In [11]:

import queue


def GetChromeBrowser(isHeadless=True):
    opt = Options()
    if isHeadless:
        opt.add_argument("--headless")
    opt.add_argument("--mute-audio")
    opt.add_argument("--disable-notifications")
    capabilities = DesiredCapabilities.CHROME
    capabilities["goog:loggingPrefs"] = {"performance": "ALL"}
    try:
        return Chrome(service=Service(ChromeDriverManager().install()), options=opt)
    except Exception as err:
        print(err)


def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ['enable-automation'])
    options.add_experimental_option("useAutomationExtension", False)

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--mute-audio")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-gpu")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)


In [ ]:
resultQ = queue.Queue()
errorQ = queue.Queue()
totalCount = 1377 * 15


def Scrapper(driver):
    global totalCount
    pageId = 1377
    totalGptsXpath = '//p[contains(text(), "We have found")]'
    linkXpath = '//a[@class="link card-title w-48"]/@href'

    while True:
        next = f"//a[contains(@href, 'page={pageId + 1}')]"
        try:
            cc_element = WebDriverWait(driver, 5).until(
                EC.visibility_of_all_elements_located((By.XPATH, totalGptsXpath)))
        except:
            pass

        try:
            cc_element = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.XPATH, linkXpath)))
        except Exception as e:
            pass

        page_source = driver.page_source
        doc = html.fromstring(page_source)

        urls = doc.xpath(linkXpath)
        count = int(''.join(doc.xpath('//p[contains(text(), "We have found")]/text()')) \
                    .replace('We have found', '') \
                    .replace('GPTs', '').strip())

        totalCount += len(urls)
        with open(f'pages/{pageId}.txt', 'wt') as file:
            file.writelines('\n'.join(urls))

        if totalCount < count:
            try:
                cc_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.XPATH, next)))
                cc_element.click()
                pageId += 1
            except Exception as e:
                pass
        else:
            break


url = f'https://gptstore.ai/gpts?page=1377'
driver = GetEdgeBrowser(False)
driver.get(url)
Scrapper(driver)


In [ ]:


title = '//h1/text()'
description = '//p[@class="leading-6 text-gray-500"]/text()'

welcome_msg = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Welcome message")]/following-sibling::dd/descendant::text()'

author = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::span/text()'
href = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::a/@href'

features = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Features and Functions")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

prompts = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Prompt Starters")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

related = '//ul[contains(@class, "grid gap-2 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-3")]/li'

for i in []:
    url = i.xpath('descendant::a/@href')
    title = i.xpath('descendant::a/text()')
    desc = i.xpath('descendant::p/text()')



In [17]:
import os

# Directory path containing text files
directory_path = f'{os.getcwd()}/pages'

# List all files in the directory
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]
print(len(files))
# Combine content of text files
combined_content = ""
for file in files:
    file_path = os.path.join(directory_path, file)
    with open(file_path, 'r', encoding='utf-8') as current_file:
        combined_content += current_file.read() + '\n'  # Add a newline between file contents

# Specify the output file path
output_file_path = f'{os.getcwd()}/combines.txt'

# Write the combined content to a new file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    output_file.write(combined_content)


1377


In [31]:

import json
import queue
import threading

from lxml import html
from selenium.webdriver import Chrome, Edge, ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager



def GetEdgeBrowser(isHeadless=True):
    options = EdgeOptions()
    if isHeadless:
        options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ['enable-automation'])
    options.add_experimental_option("useAutomationExtension", False)

    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--mute-audio")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-gpu")

    try:
        return Edge(service=Service(EdgeChromiumDriverManager().install()), options=options)
    except Exception as err:
        print(err)

def Scrapper(driver, url, name):
    driver.execute_script(f"window.open('{url}', '_blank')")
    # driver.get(url)

    title = '//h1/text()'
    description = '//p[@class="leading-6 text-gray-500"]/text()'

    welcome_msg = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Welcome message")]/following-sibling::dd/descendant::text()'

    author = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::span/text()'
    href = '//dt[contains(@class,"text-sm font-medium leading-6") and contains(text(),"Author")]/following-sibling::dd/descendant::a/@href'

    features = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Features and Functions")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

    prompts = '//dt[contains(@class, "text-xl font-medium leading-6") and h2[contains(text(), "Prompt Starters")]]/following-sibling::dd/descendant::span[contains(@class,"font-medium")]/descendant::text()'

    try:
        driver.switch_to.frame(0)
        cc_element = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="challenge-stage"]/div/label/input')))

        actions = ActionChains(driver)
        actions.move_to_element(cc_element).click().perform()
        cc_element.click()
    except Exception as e:
        pass

    try:
        cc_element = WebDriverWait(driver, 5).until(
            EC.visibility_of_all_elements_located((By.XPATH, title)))
    except Exception as e:
        pass

    d_data = {}
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight * .2);")

    page_source = driver.page_source
    doc = html.fromstring(page_source)

    d_data['title'] = '\n'.join(doc.xpath(title))
    d_data['description'] = '\n'.join(doc.xpath(description))
    d_data['welcome_msg'] = '\n'.join(doc.xpath(welcome_msg))
    d_data['author'] = '\n'.join(doc.xpath(author))
    d_data['href'] = '\n'.join(doc.xpath(href))
    d_data['features'] = '\n'.join(doc.xpath(features))
    d_data['prompts'] = '\n'.join(doc.xpath(prompts))

    relatedXpath = '//ul[contains(@class, "grid gap-2 sm:grid-cols-1 md:grid-cols-2 lg:grid-cols-3")]/li'
    related = doc.xpath(relatedXpath)

    try:
        cc_element = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, relatedXpath)))
        cc_element.click()
    except:
        pass

    rData = []
    for i in related:
        d = {'url': '\n'.join(i.xpath('descendant::a/@href')), 'title': '\n'.join(i.xpath('descendant::a/text()')),
             'desc': '\n'.join(i.xpath('descendant::p/text()'))}
        rData.append(d)

    d_data['related'] = rData

    with open(f'data/{name}.json', 'w') as json_file:
        json.dump(d_data, json_file, indent=4)

    p = input('pause')

jobs = queue.Queue()

with open(output_file_path, 'r', encoding='utf-8') as combin_file:
    urls = combin_file.read().split('\n')[:3]

for i in urls:
    jobs.put((f"https://gptstore.ai/{i}", i.split('/')[-1]))


def WorkerLoop(progress_bar: tqdm):
    driver = GetEdgeBrowser(False)
    driver.get('https://gptstore.ai/gpts?page=99')
    while not jobs.empty():
        url, name = jobs.get()
        try:
            Scrapper(driver, url, name)
        except:
            pass
        progress_bar.update(1)


threads = []
pbar = tqdm(total=len(urls))

for worker in range(1):
    thread = threading.Thread(target=WorkerLoop, args=(pbar,))
    thread.start()
    threads.append(thread)

for t in threads:
    t.join()




100%|██████████| 3/3 [01:04<00:00, 20.00s/it]

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('--disable-extensions')
options.add_argument('--no-sandbox')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')
options.add_argument('--disable-gpu')

driver = webdriver.Chrome(options=options)

driver.execute_script("window.open('https://nowsecure.nl/', '_blank')")
time.sleep(15)
driver.switch_to.window(driver.window_handles[1])

driver.switch_to.frame(0)

driver.find_element(By.XPATH, '//*[@id="challenge-stage"]/div/label/input').click()

NoSuchDriverException: Message: Unable to obtain chromedriver using Selenium Manager; Message: Unsuccessful command executed: D:\YouTubeTranscriptDownloader\venv\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --output json.
The chromedriver version cannot be discovered
; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location
